## Incorporazioni

Nel nostro esempio precedente, abbiamo lavorato su vettori bag-of-words ad alta dimensionalità con lunghezza `vocab_size`, e abbiamo esplicitamente convertito i vettori di rappresentazione posizionale a bassa dimensionalità in rappresentazioni sparse one-hot. Questa rappresentazione one-hot non è efficiente in termini di memoria. Inoltre, ogni parola viene trattata indipendentemente dalle altre, quindi i vettori codificati one-hot non esprimono somiglianze semantiche tra le parole.

In questa unità, continueremo a esplorare il dataset **News AG**. Per iniziare, carichiamo i dati e recuperiamo alcune definizioni dall'unità precedente.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Cos'è un embedding?

L'idea di **embedding** è rappresentare le parole utilizzando vettori densi a dimensione ridotta che riflettono il significato semantico della parola. Più avanti discuteremo come costruire embedding di parole significativi, ma per ora pensiamo agli embedding semplicemente come un modo per ridurre la dimensionalità di un vettore di parole.

Quindi, un livello di embedding prende una parola come input e produce un vettore di output con una dimensione specificata `embedding_size`. In un certo senso, è molto simile a un livello `Dense`, ma invece di prendere un vettore codificato one-hot come input, è in grado di accettare un numero che rappresenta la parola.

Utilizzando un livello di embedding come primo livello nella nostra rete, possiamo passare da un modello bag-of-words a un modello **embedding bag**, dove prima convertiamo ogni parola del nostro testo nel corrispondente embedding e poi calcoliamo una funzione aggregata su tutti questi embedding, come `sum`, `average` o `max`.

![Immagine che mostra un classificatore con embedding per cinque parole di una sequenza.](../../../../../translated_images/it/embedding-classifier-example.b77f021a7ee67eee.webp)

La nostra rete neurale classificatrice è composta dai seguenti livelli:

* Livello `TextVectorization`, che prende una stringa come input e produce un tensore di numeri tokenizzati. Specificheremo una dimensione ragionevole per il vocabolario `vocab_size` e ignoreremo le parole usate meno frequentemente. La forma dell'input sarà 1, e la forma dell'output sarà $n$, poiché otterremo $n$ token come risultato, ognuno contenente numeri da 0 a `vocab_size`.
* Livello `Embedding`, che prende $n$ numeri e riduce ciascun numero a un vettore denso di una lunghezza specificata (100 nel nostro esempio). Pertanto, il tensore di input di forma $n$ sarà trasformato in un tensore $n\times 100$.
* Livello di aggregazione, che calcola la media di questo tensore lungo il primo asse, ovvero calcolerà la media di tutti i $n$ tensori di input corrispondenti a parole diverse. Per implementare questo livello, utilizzeremo un livello `Lambda` e gli passeremo la funzione per calcolare la media. L'output avrà una forma di 100 e sarà la rappresentazione numerica dell'intera sequenza di input.
* Classificatore lineare finale `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Nel riepilogo stampato, nella colonna **forma dell'output**, la prima dimensione del tensore `None` corrisponde alla dimensione del minibatch, e la seconda corrisponde alla lunghezza della sequenza di token. Tutte le sequenze di token nel minibatch hanno lunghezze diverse. Discuteremo come gestirle nella prossima sezione.

Ora, alleniamo la rete:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Nota** che stiamo costruendo il vettorizzatore basandoci su un sottoinsieme dei dati. Questo viene fatto per velocizzare il processo e potrebbe comportare una situazione in cui non tutti i token del nostro testo siano presenti nel vocabolario. In questo caso, quei token verrebbero ignorati, il che potrebbe comportare una precisione leggermente inferiore. Tuttavia, nella vita reale un sottoinsieme di testo spesso fornisce una buona stima del vocabolario.


### Gestire dimensioni variabili delle sequenze

Cerchiamo di capire come avviene l'addestramento nei minibatch. Nell'esempio sopra, il tensore di input ha dimensione 1 e utilizziamo minibatch di lunghezza 128, quindi la dimensione effettiva del tensore è $128 \times 1$. Tuttavia, il numero di token in ogni frase è diverso. Se applichiamo il livello `TextVectorization` a un singolo input, il numero di token restituiti varia a seconda di come il testo viene tokenizzato:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Tuttavia, quando applichiamo il vettorizzatore a diverse sequenze, deve produrre un tensore di forma rettangolare, quindi riempie gli elementi non utilizzati con il token PAD (che nel nostro caso è zero):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Qui possiamo vedere gli embeddings:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Nota**: Per ridurre al minimo la quantità di padding, in alcuni casi ha senso ordinare tutte le sequenze nel dataset in ordine di lunghezza crescente (o, più precisamente, in base al numero di token). Questo garantirà che ogni minibatch contenga sequenze di lunghezza simile.


## Integrazioni semantiche: Word2Vec

Nel nostro esempio precedente, il livello di embedding ha imparato a mappare le parole in rappresentazioni vettoriali, tuttavia queste rappresentazioni non avevano un significato semantico. Sarebbe utile apprendere una rappresentazione vettoriale in cui parole simili o sinonimi corrispondano a vettori vicini tra loro in termini di una certa distanza vettoriale (ad esempio la distanza euclidea).

Per fare ciò, dobbiamo pre-addestrare il nostro modello di embedding su una vasta collezione di testi utilizzando una tecnica come [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Si basa su due principali architetture utilizzate per produrre una rappresentazione distribuita delle parole:

 - **Continuous bag-of-words** (CBoW), dove si addestra il modello a prevedere una parola dal contesto circostante. Dato l'ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, l'obiettivo del modello è prevedere $W_0$ a partire da $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram**, che è l'opposto del CBoW. Il modello utilizza la finestra di parole di contesto circostanti per prevedere la parola corrente.

CBoW è più veloce, mentre skip-gram, pur essendo più lento, rappresenta meglio le parole meno frequenti.

![Immagine che mostra entrambi gli algoritmi CBoW e Skip-Gram per convertire le parole in vettori.](../../../../../translated_images/it/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Per sperimentare con l'embedding Word2Vec pre-addestrato sul dataset di Google News, possiamo utilizzare la libreria **gensim**. Di seguito troviamo le parole più simili a 'neural'.

> **Nota:** Quando si creano per la prima volta i vettori delle parole, il download può richiedere un po' di tempo!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Possiamo anche estrarre l'incorporamento vettoriale dalla parola, da utilizzare nell'addestramento del modello di classificazione. L'incorporamento ha 300 componenti, ma qui mostriamo solo le prime 20 componenti del vettore per chiarezza:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

La cosa straordinaria delle incorporazioni semantiche è che puoi manipolare la codifica vettoriale basata sulla semantica. Ad esempio, possiamo chiedere di trovare una parola la cui rappresentazione vettoriale sia il più vicino possibile alle parole *re* e *donna*, e il più lontano possibile dalla parola *uomo*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Un esempio sopra utilizza una magia interna di GenSym, ma la logica sottostante è in realtà abbastanza semplice. Una cosa interessante sugli embedding è che si possono eseguire operazioni vettoriali normali sui vettori di embedding, e ciò rifletterebbe operazioni sui **significati** delle parole. L'esempio sopra può essere espresso in termini di operazioni vettoriali: calcoliamo il vettore corrispondente a **RE-UOMO+DONNA** (le operazioni `+` e `-` vengono eseguite sulle rappresentazioni vettoriali delle parole corrispondenti), e poi troviamo la parola più vicina nel dizionario a quel vettore:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTA**: Abbiamo dovuto aggiungere piccoli coefficienti ai vettori *man* e *woman* - prova a rimuoverli per vedere cosa succede.

Per trovare il vettore più vicino, utilizziamo la struttura di TensorFlow per calcolare un vettore di distanze tra il nostro vettore e tutti i vettori nel vocabolario, e poi troviamo l'indice della parola minima usando `argmin`.


Sebbene Word2Vec sembri un ottimo metodo per esprimere la semantica delle parole, presenta numerosi svantaggi, tra cui i seguenti:

* Sia i modelli CBoW che skip-gram sono **embedding predittivi**, e considerano solo il contesto locale. Word2Vec non sfrutta il contesto globale.
* Word2Vec non tiene conto della **morfologia** delle parole, ovvero il fatto che il significato di una parola possa dipendere da diverse parti della parola, come la radice.

**FastText** cerca di superare questa seconda limitazione, e si basa su Word2Vec imparando rappresentazioni vettoriali per ogni parola e per i n-grammi di caratteri presenti all'interno di ciascuna parola. I valori delle rappresentazioni vengono poi mediati in un unico vettore a ogni passo di allenamento. Sebbene ciò aggiunga molta computazione extra durante il pretraining, consente agli embedding di parole di codificare informazioni relative alle sotto-parole.

Un altro metodo, **GloVe**, utilizza un approccio diverso per gli embedding di parole, basato sulla fattorizzazione della matrice parola-contesto. Per prima cosa, costruisce una grande matrice che conta il numero di occorrenze delle parole in diversi contesti, e poi cerca di rappresentare questa matrice in dimensioni inferiori in modo da minimizzare la perdita di ricostruzione.

La libreria gensim supporta questi embedding di parole, e puoi sperimentarli modificando il codice di caricamento del modello sopra.


## Utilizzo di embedding pre-addestrati in Keras

Possiamo modificare l'esempio sopra per prepopolare la matrice nel nostro livello di embedding con embedding semantici, come Word2Vec. I vocabolari dell'embedding pre-addestrato e del corpus di testo probabilmente non corrisponderanno, quindi dobbiamo sceglierne uno. Qui esploriamo le due opzioni possibili: utilizzare il vocabolario del tokenizer e utilizzare il vocabolario degli embedding di Word2Vec.

### Utilizzo del vocabolario del tokenizer

Quando si utilizza il vocabolario del tokenizer, alcune parole del vocabolario avranno embedding Word2Vec corrispondenti, mentre altre saranno mancanti. Dato che la dimensione del nostro vocabolario è `vocab_size`, e la lunghezza del vettore di embedding Word2Vec è `embed_size`, il livello di embedding sarà rappresentato da una matrice di pesi di forma `vocab_size`$\times$`embed_size`. Popoleremo questa matrice passando attraverso il vocabolario:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Per le parole che non sono presenti nel vocabolario di Word2Vec, possiamo lasciarle come zeri oppure generare un vettore casuale.

Ora possiamo definire un livello di embedding con pesi pre-addestrati:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Nota**: Notate che abbiamo impostato `trainable=False` durante la creazione di `Embedding`, il che significa che non stiamo riaddestrando il livello Embedding. Questo potrebbe causare una leggera diminuzione della precisione, ma accelera l'addestramento.

### Utilizzo del vocabolario di embedding

Un problema con l'approccio precedente è che i vocabolari utilizzati in TextVectorization ed Embedding sono diversi. Per superare questo problema, possiamo utilizzare una delle seguenti soluzioni:
* Riaddestrare il modello Word2Vec sul nostro vocabolario.
* Caricare il nostro dataset utilizzando il vocabolario del modello Word2Vec preaddestrato. I vocabolari utilizzati per caricare il dataset possono essere specificati durante il caricamento.

Il secondo approccio sembra più semplice, quindi implementiamolo. Per prima cosa, creeremo un livello `TextVectorization` con il vocabolario specificato, preso dagli embedding di Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

La libreria di word embeddings gensim contiene una funzione comoda, `get_keras_embeddings`, che creerà automaticamente il corrispondente livello di embeddings di Keras per te.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Uno dei motivi per cui non stiamo ottenendo una maggiore accuratezza è che alcune parole del nostro dataset mancano nel vocabolario GloVe preaddestrato e quindi vengono essenzialmente ignorate. Per superare questo problema, possiamo addestrare i nostri embedding basati sul nostro dataset.


## Contesti contestuali

Una delle principali limitazioni delle rappresentazioni di embedding pre-addestrati tradizionali, come Word2Vec, è il fatto che, anche se possono catturare parte del significato di una parola, non riescono a distinguere tra significati diversi. Questo può causare problemi nei modelli a valle.

Ad esempio, la parola 'play' ha significati diversi in queste due frasi:
- Sono andato a vedere una **play** a teatro.
- John vuole **play** con i suoi amici.

Gli embedding pre-addestrati di cui abbiamo parlato rappresentano entrambi i significati della parola 'play' nello stesso embedding. Per superare questa limitazione, dobbiamo costruire embedding basati sul **modello linguistico**, che è addestrato su un ampio corpus di testo e *sa* come le parole possono essere combinate in contesti diversi. Discutere degli embedding contestuali esula dallo scopo di questo tutorial, ma ne parleremo quando affronteremo i modelli linguistici nella prossima unità.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
